In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4905/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4905/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_1_2,0.944336,0.708256,0.616135,0.918375,0.222983,1.950895,1.798028,0.190970,2.548433,0.472211,1.148436,0.492314,69.001320,109.224222,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_1_6,0.947635,0.706296,0.623788,0.916207,0.209770,1.964000,1.762181,0.196043,2.455713,0.458007,1.139641,0.477505,69.123486,109.346388,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_1_9,0.949861,0.704414,0.627952,0.913986,0.200850,1.976585,1.742679,0.201240,2.387990,0.448163,1.133703,0.467242,69.210392,109.433294,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_0_7,0.919863,0.696060,0.540180,0.947544,0.321020,2.032446,2.153803,0.323913,3.127890,0.566586,1.213698,0.590707,68.272505,108.495407,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_0_3,0.912837,0.695507,0.526103,0.942128,0.349168,2.036149,2.219741,0.357356,3.250450,0.590904,1.232435,0.616060,68.104406,108.327308,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,model_6_12_2,0.988381,0.126707,-1.092854,0.917130,0.046545,5.839714,9.802961,0.095975,0.126497,0.215742,1.278856,0.224927,56.134686,86.606582,"Hidden Size=[4, 2], regularizer=0.02, learning..."
843,model_6_12_6,0.989251,0.125202,-1.102275,0.923363,0.043061,5.849779,9.847088,0.088757,0.120566,0.207511,1.257983,0.216345,56.290288,86.762184,"Hidden Size=[4, 2], regularizer=0.02, learning..."
844,model_6_12_9,0.989750,0.123051,-1.109572,0.925843,0.041060,5.864162,9.881266,0.085884,0.115808,0.202634,1.245999,0.211260,56.385425,86.857321,"Hidden Size=[4, 2], regularizer=0.02, learning..."
845,model_6_26_0,0.996787,0.122694,-1.218478,0.993877,0.012871,5.866546,10.391385,0.031716,0.192247,0.113449,1.077110,0.118278,58.705624,89.177520,"Hidden Size=[4, 2], regularizer=0.02, learning..."
